# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [22]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine, inspect

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split


In [30]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
inspector = inspect(engine)
print(inspector.get_table_names(),'\n')

df = pd.read_sql_table('Messages', engine)
print(df.head())
print('Columns in the dataframe:')
for col in df.columns:
    print(f'{col}')

['Messages'] 

   id                                            message   genre  related  \
0   2  Weather update - a cold front from Cuba that c...  direct        1   
1   7            Is the Hurricane over or is it not over  direct        1   
2   8                    Looking for someone but no name  direct        1   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  12  says: west side of Haiti, rest of the country ...  direct        1   

   request  offer  aid_related  medical_help  medical_products  \
0        0      0            0             0                 0   
1        0      0            1             0                 0   
2        0      0            0             0                 0   
3        1      0            1             0                 1   
4        0      0            0             0                 0   

   search_and_rescue  ...  aid_centers  other_infrastructure  weather_related  \
0                  0  ...            0      

In [24]:
# define feature variable X (the message text)
X = df['message']

# define target variable Y (the categories) and drop columns that are not target variables: 'id', 'message', 'genre'
Y = df.drop(columns=['id', 'message', 'genre'])

print(X.shape)
print(Y.shape)

(26216,)
(26216, 36)


### 2. Write a tokenization function to process your text data

In [25]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

def tokenize(text):
    """
    Tokenization function to process text data.

    Inputs:
    text(str): The input text to be tokenized and processed.

    Returns:
    list: A list of clean, processed tokens (words) from the input text.
    """
    # normalize text: convert to lowercase and remove punctuation
    text = text.lower() # convert text to lowercase to standardize it
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) # replace non-alphanumeric characters with spaces

    # tokenize the text into words
    tokens = word_tokenize(text) # split the text into individual words (tokens)

    # remove stopwords (common words that don't add much meaning)
    tokens = [word for word in tokens if word not in stopwords.words('english')]

    # lemmatize: convert words to their base form
    lemmatizer = WordNetLemmatizer() # initialize the lemmatizer
    clean_tokens = [lemmatizer.lemmatize(token) for token in tokens] # lematize each token

    return clean_tokens

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/leylaalieva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leylaalieva/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leylaalieva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [26]:
# build the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [27]:
# split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# train the pipeline
pipeline.fit(X_train, Y_train)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x134359b20>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [28]:
# predict on the test set
Y_pred = pipeline.predict(X_test)

# print classification report for each target category
for i, col in enumerate(Y_test.columns):
    print(f"Category: {col}")
    print(classification_report(Y_test[col], Y_pred[:,i]))

Category: related
              precision    recall  f1-score   support

           0       0.70      0.42      0.52      1266
           1       0.83      0.94      0.88      3938
           2       0.47      0.38      0.42        40

    accuracy                           0.81      5244
   macro avg       0.67      0.58      0.61      5244
weighted avg       0.80      0.81      0.79      5244

Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.83      0.48      0.61       895

    accuracy                           0.90      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.90      0.88      5244

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.5

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5244

    accuracy                           1.00      5244
   macro avg       1.00      1.00      1.00      5244
weighted avg       1.00      1.00      1.00      5244

Category: water
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4905
           1       0.85      0.40      0.55       339

    accuracy                           0.96      5244
   macro avg       0.91      0.70      0.76      5244
weighted avg       0.95      0.96      0.95      5244

Category: food
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4649
           1       0.86      0.59      0.70       595

    accuracy                           0.94      5244
   macro avg       0.90      0.79      0.84      5244
weighted avg       0.94      0.94      0.94      5244

Category: shelter
              precision 

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-pa

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5212
           1       0.00      0.00      0.00        32

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      0.99      0.99      5244

Category: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5198
           1       0.00      0.00      0.00        46

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.98      0.99      0.99      5244

Category: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5222
           1       0.00      0.00      0.00        22

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00     

The classification report above shows that in many categories (e.g. offer, shops, tools) the number of messages labeled as 1 is extremeky small  compared to those labeled as 0. This leads to poor performance in those categories as the model has difficulty learning from such imbalanced data.<br>
Categories like related, request and aid_related, which have more balanced distributions of 0s and 1s, show much better precision, recall and F1-scores.<br>

I'd therefore address the class imbalance first. I will explore the techniques like oversampling the minority class or undersampling the majority class to balance the data.

### 6. Improve your model
Use grid search to find better parameters. 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.